<a href="https://colab.research.google.com/github/AAhmadS/NLP_HW4/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Natural Language Processing-Homework4**

####**Students' names** :
>Amirahmad Shafiee<br/>
>Ali Shafiee<br/>
>Neda Fallah<br/>
####**Student number** : 99104027<br/>**Chosen task** : classification In this notebook we aim to do common but rather essential preprocess actions on some text chosen from the persian literature world. After doing so, an NLP task would be operated on the processed text.

In [1]:
!pip install -qU transformers
!pip install -qU torchtext
!pip install -qU fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
!git clone https://github.com/facebookresearch/fastText.git

Cloning into 'fastText'...
remote: Enumerating objects: 3946, done.
remote: Counting objects: 100% (980/980), done.
remote: Compressing objects: 100% (151/151), done.
remote: Total 3946 (delta 878), reused 839 (delta 828), pack-reused 2966
Receiving objects: 100% (3946/3946), 8.25 MiB | 14.65 MiB/s, done.
Resolving deltas: 100% (2510/2510), done.


In [3]:
import fasttext.util

In [4]:
!sudo pip install fastText/.
!cd fastText
fasttext.util.download_model('fa', if_exists='ignore')
ft = fasttext.load_model('cc.fa.300.bin')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./fastText
  Preparing metadata (setup.py) ... done
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4384288 sha256=21dc9db8052fec4d9a09638cc204b9da324b7ef7d72c8c350d22d896078f94ae
  Stored in directory: /tmp/pip-ephem-wheel-cache-xy5o13wr/wheels/8b/05/af/3cfae069d904597d44b309c956601b611bdf8967bcbe968903
Successfully built fasttext
  Attempting uninstall: fasttext
    Found existing installation: fasttext 0.9.2
    Uninstalling fasttext-0.9.2:
      Successfully uninstalled fasttext-0.9.2



In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd /content/drive/MyDrive/nlp_hw4

/content/drive/MyDrive/nlp_hw4


##Essential packages

In [68]:
import pandas as pd
import numpy as np
from numpy import pad
import json
import matplotlib.pyplot as plt
plt.switch_backend('agg')
%matplotlib inline
import matplotlib.ticker as ticker
from collections import Counter
from __future__ import unicode_literals, print_function, division
import random
from tqdm import tqdm
from sklearn.utils import resample
from joblib import Parallel, delayed
from sklearn import metrics
from sklearn.model_selection import train_test_split as tts
import sklearn.svm as svm
from sklearn.model_selection import StratifiedKFold as skf
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score as acs
import plotly.express as px
from sklearn.metrics import roc_curve, auc,precision_recall_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.utils.fixes import loguniform
from sklearn.metrics import f1_score
from sklearn.model_selection import learning_curve
from sklearn.metrics import classification_report
import seaborn as sns
import fasttext.util

import itertools
import torch
from torch import nn
import torchtext
from torchtext.vocab import Vocab
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchtext.data.utils import get_tokenizer
import torch.nn.functional as F
from torch import optim

import nltk, re
from nltk import word_tokenize, sent_tokenize, WhitespaceTokenizer, FreqDist, pos_tag
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.probability import FreqDist


from transformers import pipeline, AutoTokenizer


import plotly.express as px
import plotly.graph_objects as go

from tqdm.notebook import tqdm

import os

import copy
import collections
import torch

##Load data and analysis


In [ ]:
with open('food.jsonl', 'r') as json_file:
    json_list = list(json_file)

data = pd.DataFrame([])
for json_str in json_list:
    result = json.loads(json_str)
    data = data.append(result,ignore_index=True)

In [36]:
data.isna().value_counts()

review  sentiment  category  aspects
False   False      False     False      1917
dtype: int64

In [58]:
data.sample(10)

,review,sentiment,category,aspects
1130,همه می دونن که عسل انواع مختلفی داره؛ البته با...,1,عسل,{'ارزش خرید': '1'}
799,هم قیمتش نسبت ب روغن های دیگه مناسب تره هم از ...,1,روغن,"{'ارزش خرید': '1', 'کیفیت': '1'}"
1693,با بقیه چایی ها هیچ فرقی نداره چه طعم چه رنگ,0,چای,{'طعم': '0'}
95,وقتی دستم رسید خیلی از بیسکوبیت ها خورد و نصف ...,-2,بیسکویت و ویفر,{}
1687,پولتونو تو چاه نریزید،کافئین کم،طعم ضعیف،متاسف...,-2,قهوه,"{'کیفیت': '-1', 'ارزش خرید': '-2', 'طعم': '-1'}"
907,ریز بودن حتی تو تخفیف هم نگیرین خوشمزه نیست اسلا,-2,تخمه و مغز طعم‌دار,{'طعم': '-2'}
184,خیلی تند هست من که زیاد نمیتونم تو دهن ام نگه ...,-2,آدامس و خوشبو کننده‌ی دهان,{'طعم': '-2'}
600,تکه های ماهی درشت هستند و مثل خیلی از تن ها خر...,1,تن ماهی,{}
373,برای مدرسه بچه ها مناسب است و چون ویتامینه است...,1,شیر,{'ارزش غذایی': '1'}
562,بطری های عالیس طوری طراحی شدند که نور به هیچ ع...,2,شیر,{'بسته بندی': '2'}


In [59]:
data["words_count"] = [len(x.split()) for x in data["review"]]

##Essential plots

In [72]:
class distributions:

  def minmax(data, col_end = 'words_count', col_beg = 'review'):
    data[col_end] = data[col_beg].apply(lambda t: len(t.split()))

    summary = data[col_end].min(), data[col_end].max(), data[col_end].mean(), np.std(data[col_end])
    print(f" max: {summary[1]} \n min :{summary[0]}\n mean : {summary[2]}\n std : {summary[3]}")

  def word_freq(data, col_end = "words_count"):

      fig = go.Figure()

      fig.add_trace(go.Histogram(
          x=data[col_end]
      ))

      fig.update_layout(
          title_text='Distribution of word counts within data',
          xaxis_title_text='Word Count',
          yaxis_title_text='Frequency',
          bargap=0.2,
          bargroupgap=0.2)

      fig.show()

  def data_gl_than(data, less_than=25.0, greater_than=10.0, col='words_count'):
      data_length = data[col].values

      data_glt = sum([1 for length in data_length if greater_than < length <= less_than])

      data_glt_rate = (data_glt / len(data_length)) * 100

      print(f'sentences with word length of greater than {greater_than} and less than {less_than} includes {data_glt_rate:.2f}% of the whole!')

  def full_df_run(data, col_begin = 'sentences', minmax = True, freq = True, gl = True):
    if minmax :
      distributions.minmax(data)
    if freq:
      distributions.word_freq(data)
    if gl:
      distributions.data_gl_than(data)

  def word_and_freq(data, col = "tokenized_sents"):
    tokenized_sent = data[col].values
    total_len = 0
    for i in range(len(tokenized_sent)):
      total_len += len(tokenized_sent[i])

    mp_freqdist = FreqDist(itertools.chain(*tokenized_sent))
    top20words=mp_freqdist.most_common(20)
    print ('%-16s' % 'word', '%-16s' % 'Frequency','%-16s' %  '% of the total')
    for topword in top20words:
        percent=(topword[1]/total_len)*100
        print ('%-16s' % topword[0], '%-16s' % topword[1],'%-16s' %  percent)

  def author_count(data, col_auth):
    fig = go.Figure()

    groupby_rate = data.groupby(col_auth)[col_auth].count()

    fig.add_trace(go.Bar(
        x=list(sorted(groupby_rate.index)),
        y=groupby_rate.tolist(),
        text=groupby_rate.tolist(),
        textposition='auto'
    ))

    fig.update_layout(
        title_text='Distribution of authors within sentences',
        xaxis_title_text='author',
        yaxis_title_text='Frequency',
        bargap=0.2,
        bargroupgap=0.2)

    fig.show()

In [73]:
distributions.full_df_run(data)

 max: 50 
 min :8
 mean : 18.62336984872196
 std : 8.860481005381581


sentences with word length of greater than 10.0 and less than 25.0 includes 70.06% of the whole!


##Document classification

###Tf-Idf

In [75]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [76]:
corpus = [text for text in data["review"]]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()


array(['10', '1000', '12', ..., '۹۰۰', '۹۵', '۹۸۰۰'], dtype=object)

###SVM linear kernel

In [ ]:
##related to scaling and splitting

y = df['y-axis-mean']
df.drop(columns={'y-axis-mean'},inplace=True)
x=df

##personally have a feeling that minmax scaler would by far be the better one

x=MinMaxScaler().fit_transform(x)
X_train , x_test , Y_train , y_test = tts(x,y,test_size = 0.1 , random_state = 42)
x_train , x_val , y_train , y_val = tts(X_train,Y_train,test_size=0.1,random_state=42)

In [ ]:
parameters = dict(kernel=['linear'],C = [5e-3,1e-2,5e-2,1e-1,5e-1,1])

####Preprocess data<br/>implement the cross validation function

####Train and evaluation section

In [ ]:
random_model = svm.SVC(max_iter=1000,probability=True,verbose=False)
model_acc = GridSearchCV(estimator=random_model.copy(),param_distributions=parameters,n_iter=6,n_jobs=-1,cv=10,scoring='accuracy',refit=True)
model_f1 = GridSearchCV(estimator=random_model.copy(),param_distributions=parameters,n_iter=6,n_jobs=-1,cv=10,scoring='f1',refit=True)
model_rac = GridSearchCV(estimator=random_model.copy(),param_distributions=parameters,n_iter=6,n_jobs=-1,cv=10,scoring='roc_auc',refit=True)

In [ ]:
print(f'score of model with score set to accuracy :{model_acc.best_score_}\nparameters of model with score set to accuracy:{model_acc.best_params_}\n-------')
print(f'score of model with score set to f1 :{model_f1.best_score_}\nparameters of model with score set to f1:{model_f1.best_params_}\n-----------')
print(f'score of model with score set to roc-auc :{model_rac.best_score_}\nparameters of model with score set to roc-auc:{model_rac.best_params_}\n----------')

In [ ]:
def roc_curve(model,title,score):
  y_score = model.predict_proba(x)[:, 1]
  fpr, tpr, thresholds = metrics.roc_curve(y, y_score)

  fig = px.area(
      x=fpr, y=tpr,
      title=f'{title}\'s ROC Curve (AUC={auc(fpr, tpr):.4f})  -- score:{score}',
      labels=dict(x='False Positive Rate', y='True Positive Rate'),
      width=700, height=500
  )
  fig.add_shape(
      type='line', line=dict(dash='dash'),
      x0=0, x1=1, y0=0, y1=1
  )

  fig.update_yaxes(scaleanchor="x", scaleratio=1)
  fig.update_xaxes(constrain='domain')
  fig.show()

roc_curve(model_acc,"Model_acc","accuracy")
roc_curve(model_f1,"Model_f1","f1")
roc_curve(model_rac,"Model_rac","roc_auc")

plt.legend()
plt.show()

In [ ]:
def prec_recall(model,title,score):

  y_score = model.predict_proba(x)[:, 1]
  precision, recall, thresholds = precision_recall_curve(y, y_score)

  fig = px.area(
      x=recall, y=precision,
      title=f'{title}\'s Precision-Recall Curve (AUC={auc(precision, recall):.4f}) -- score : {score}',
      labels=dict(x='Recall', y='Precision'),
      width=700, height=500
  )
  fig.add_shape(
      type='line', line=dict(dash='dash'),
      x0=0, x1=1, y0=1, y1=0
  )
  fig.update_yaxes(scaleanchor="x", scaleratio=1)
  fig.update_xaxes(constrain='domain')

  fig.show()


prec_recall(model_acc,"Model_acc","accuracy")
prec_recall(model_f1,"Model_f1","f1")
prec_recall(model_rac,"Model_rac","roc_auc")

plt.legend()
plt.show()

In [ ]:
def learning_curve(model,x,y,fold,title):
  train_sizes, train_scores, test_scores = learning_curve(estimator=model, X=x, y=y,cv=fold,)
  train_mean = np.mean(train_scores, axis=1)
  train_std = np.std(train_scores, axis=1)
  test_mean = np.mean(test_scores, axis=1)
  test_std = np.std(test_scores, axis=1)

  plt.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5, label='Training Accuracy')
  plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')
  plt.plot(train_sizes, test_mean, color='green', marker='+', markersize=5, linestyle='--', label='Validation Accuracy')
  plt.fill_between(train_sizes, test_mean + test_std, test_mean - test_std, alpha=0.15, color='green')
  plt.title(f'Learning Curve for {title}')
  plt.xlabel('Training Data Size')
  plt.ylabel('Model accuracy')
  plt.grid()
  plt.legend(loc='lower right')
  plt.show()


learning_curve(model_acc,X_train,Y_train,10,"Model_acc")
learning_curve(model_f1,X_train,Y_train,10,"Model_f1")
learning_curve(model_rac,X_train,Y_train,10,"Model_rac")

####Test

###Transformer-based model

In [ ]:
from transformers import BertConfig, BertTokenizer
from transformers import BertModel

from transformers import get_linear_schedule_with_warmup

import torch
import torch.nn as nn
import torch.nn.functional as F
import os

In [ ]:
train, test = tts(new_data, test_size=0.2, random_state=1, stratify=new_data['author'])
train, valid = tts(train, test_size=0.2, random_state=1, stratify=train['author'])

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

x_train, y_train = train['normalized_sents'].values.tolist(), train['author'].values.tolist()
x_valid, y_valid = valid['normalized_sents'].values.tolist(), valid['author'].values.tolist()
x_test, y_test = test['normalized_sents'].values.tolist(), test['author'].values.tolist()

print(train.shape)
print(valid.shape)
print(test.shape)

(8662, 3)
(2166, 3)
(2708, 3)


##### Configuration

In [ ]:
##sit the model on the gpu

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device: cpu
CUDA is not available.  Training on CPU ...


In [ ]:
# general config
##to be changed....


MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16

EPOCHS = 3
EEVERY_EPOCH = 1000
LEARNING_RATE = 2e-5
CLIP = 0.0

MODEL_NAME_OR_PATH = 'bert-base-uncased'
OUTPUT_PATH = '/auth_classification_bert/pytorch_model.bin'

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

In [ ]:
# create a key finder based on label 2 id and id to label
##tokenizing the labels in a handy way

label2id = {label: i for i, label in enumerate(name_list)}
id2label = {v: k for k, v in label2id.items()}

print(f'label2id: {label2id}')
print(f'id2label: {id2label}')

label2id: {'heg': 0, 'kan': 1, 'nei': 2}
id2label: {0: 'heg', 1: 'kan', 2: 'nei'}


In [ ]:
# setup the tokenizer and configuration

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config = BertConfig.from_pretrained(
    MODEL_NAME_OR_PATH, **{
        'label2id': label2id,
        'id2label': id2label,
    })

print(config.to_json_string())

{
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "heg",
    "1": "kan",
    "2": "nei"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "heg": 0,
    "kan": 1,
    "nei": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



##### Input Embeddings

In [ ]:
idx = np.random.randint(0, len(train))
sample_sent = train.iloc[idx]['normalized_sents']
sample_auth = train.iloc[idx]['author']

print(f'Sample: \n{" ".join(sample_sent)}\n{sample_auth}')

Sample: 
proof one perhaps say immediately con scious representation external things consequently still remains undecided whether something outside corresponding
kan


In [ ]:
tokens = sample_sent
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f'  sentence: {" ".join(sample_sent)}')
print(f'  Coded_tokens: {tokens}')
print(f'   Tokens: {tokenizer.convert_tokens_to_string(tokens)}')
print(f'Token IDs: {token_ids}')

  sentence: proof one perhaps say immediately con scious representation external things consequently still remains undecided whether something outside corresponding
  Coded_tokens: ['proof', 'one', 'perhaps', 'say', 'immediately', 'con', 'scious', 'representation', 'external', 'things', 'consequently', 'still', 'remains', 'undecided', 'whether', 'something', 'outside', 'corresponding']
   Tokens: proof one perhaps say immediately con scious representation external things consequently still remains undecided whether something outside corresponding
Token IDs: [6947, 2028, 3383, 2360, 3202, 9530, 100, 6630, 6327, 2477, 8821, 2145, 3464, 100, 3251, 2242, 2648, 7978]


In [ ]:
encoding = tokenizer.encode_plus(
    sample_sent,
    max_length=32,
    truncation=True,
    add_special_tokens=True, # Add '[CLS]' and '[SEP]'
    return_token_type_ids=True,
    return_attention_mask=True,
    padding='max_length',
    return_tensors='pt',  # Return PyTorch tensors
)

print(f'Keys: {encoding.keys()}\n')
for k in encoding.keys():
    print(f'{k}:\n{encoding[k]}')

Keys: dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

input_ids:
tensor([[ 101, 6947, 2028, 3383, 2360, 3202, 9530,  100, 6630, 6327, 2477, 8821,
         2145, 3464,  100, 3251, 2242, 2648, 7978,  102,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]])
token_type_ids:
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])


##### Dataset

In [ ]:
class PhilDataset(torch.utils.data.Dataset):

    def __init__(self, tokenizer, sent, targets=None, label_list=None, max_len=128):
        self.sent = sent
        self.targets = targets
        self.has_target = isinstance(targets, list) or isinstance(targets, np.ndarray)

        self.tokenizer = tokenizer
        self.max_len = max_len


        self.label_map = {label: i for i, label in enumerate(label_list)} if isinstance(label_list, list) else {}

    def __len__(self):
        return len(self.sent)

    def __getitem__(self, item):
        sentence = str(self.sent[item])

        if self.has_target:
            target = self.targets[item]

        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')

        inputs = {
            'sentence': sentence,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }

        if self.has_target:
            inputs['targets'] = torch.tensor(target, dtype=torch.long)

        return inputs


def create_data_loader(x, y, tokenizer, max_len, batch_size, label_list):
    dataset = PhilDataset(
        sent=x,
        targets=y,
        tokenizer=tokenizer,
        max_len=max_len,
        label_list=label_list)

    return torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [ ]:
label_list = name_list
train_data_loader = create_data_loader(train['normalized_sents'].to_numpy(), train['labels'].to_numpy(), tokenizer, MAX_LEN, TRAIN_BATCH_SIZE, label_list)
valid_data_loader = create_data_loader(valid['normalized_sents'].to_numpy(), valid['labels'].to_numpy(), tokenizer, MAX_LEN, VALID_BATCH_SIZE, label_list)
test_data_loader = create_data_loader(test['normalized_sents'].to_numpy(), None, tokenizer, MAX_LEN, TEST_BATCH_SIZE, label_list)

In [ ]:
sample_data = next(iter(train_data_loader))

print(sample_data.keys())

print(sample_data['sentence'])
print(sample_data['input_ids'].shape)
print(sample_data['input_ids'][0, :])
print(sample_data['attention_mask'].shape)
print(sample_data['attention_mask'][0, :])
print(sample_data['token_type_ids'].shape)
print(sample_data['token_type_ids'][0, :])
print(sample_data['targets'].shape)
print(sample_data['targets'])

dict_keys(['sentence', 'input_ids', 'attention_mask', 'token_type_ids', 'targets'])
["['employ', 'innovations', 'quaint', 'old', 'terms', 'language', 'favour', 'rare', 'strange', 'concerned', 'aug\\xad', 'ment', 'vocabulary', 'rather', 'restrict', 'always', 'sign', 'immature', 'corrupted', 'taste']", "['notion', 'consciousness', 'forms', 'absolute', 'interfusion', 'individuality', 'let', 'see', 'whether', 'notion', 'confirmed', 'experience', 'whether', 'reality', 'corresponds']", "['one', 'indi\\xad', 'vidual', 'needs', 'health', 'another', 'cause', 'sickness', 'many', 'ways', 'means', 'freedom', 'spirit', 'may', 'highly', 'developed', 'natures', 'count', 'ways', 'means', 'unfreedom']", "['deceiving', 'priesthodd', 'dppressive', 'despot', 'therefore', 'ndt', 'directly', 'dbject', 'activity', 'object', 'insight', 'devoid']", "['individual', 'rules', 'life', 'excite', 'hostility', 'adopts', 'people', 'feel', 'humiliated', 'exceptional', 'treatment', 'accords', 'though', 'treated', 'merel

In [ ]:
sample_test = next(iter(test_data_loader))
print(sample_test.keys())

dict_keys(['sentence', 'input_ids', 'attention_mask', 'token_type_ids'])


In [ ]:
class ClassifierModel(nn.Module):

    def __init__(self, config):
        super(ClassifierModel, self).__init__()

        self.bert = BertModel.from_pretrained(MODEL_NAME_OR_PATH)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, 10)

    def forward(self, input_ids, attention_mask, token_type_ids):
        pooled_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)['pooler_output']

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

In [ ]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()
pt_model = None

In [ ]:
pt_model = ClassifierModel(config=config)
pt_model = pt_model.to(device)

print('pt_model', type(pt_model))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


pt_model <class '__main__.ClassifierModel'>


In [ ]:
for name,param in pt_model.named_parameters():
    print(name)

##### Training

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

import plotly.express as px
import plotly.graph_objects as go

from tqdm.notebook import tqdm

import os

import copy
import collections
import torch

In [ ]:
def simple_accuracy(y_true, y_pred):
    return (y_true == y_pred).mean()

def acc_and_f1(y_true, y_pred, average='weighted'):
    acc = simple_accuracy(y_true, y_pred)
    f1 = f1_score(y_true=y_true, y_pred=y_pred, average=average)
    return {
        "acc": acc,
        "f1": f1,
    }

def y_loss(y_true, y_pred, losses):
    y_true = torch.stack(y_true).cpu().detach().numpy()
    y_pred = torch.stack(y_pred).cpu().detach().numpy()
    y = [y_true, y_pred]
    loss = np.mean(losses)

    return y, loss


def eval_op(model, data_loader, loss_fn):
    model.eval()

    losses = []
    y_pred = []
    y_true = []

    with torch.no_grad():
        for dl in tqdm(data_loader, total=len(data_loader), desc="Evaluation... "):

            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']
            targets = dl['targets']

            # move tensors to GPU if CUDA is available
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            targets = targets.to(device)

            # compute predicted outputs by passing inputs to the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)

            # convert output probabilities to predicted class
            _, preds = torch.max(outputs, dim=1)

            # calculate the batch loss
            loss = loss_fn(outputs, targets)

            # accumulate all the losses
            losses.append(loss.item())

            y_pred.extend(preds)
            y_true.extend(targets)

    eval_y, eval_loss = y_loss(y_true, y_pred, losses)
    return eval_y, eval_loss


def train_op(model,
             data_loader,
             loss_fn,
             optimizer,
             scheduler,
             step=0,
             print_every_step=100,
             eval=False,
             eval_cb=None,
             eval_loss_min=np.Inf,
             eval_data_loader=None,
             clip=0.0):

    model.train()

    losses = []
    y_pred = []
    y_true = []

    for dl in tqdm(data_loader, total=len(data_loader), desc="Training... "):
        step += 1

        input_ids = dl['input_ids']
        attention_mask = dl['attention_mask']
        token_type_ids = dl['token_type_ids']
        targets = dl['targets']

        # move tensors to GPU if CUDA is available
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        targets = targets.to(device)

        # clear the gradients of all optimized variables
        optimizer.zero_grad()

        # compute predicted outputs by passing inputs to the model
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)

        # convert output probabilities to predicted class
        _, preds = torch.max(outputs, dim=1)

        # calculate the batch loss
        loss = loss_fn(outputs, targets)

        # accumulate all the losses
        losses.append(loss.item())

        # compute gradient of the loss with respect to model parameters
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        if clip > 0.0:
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)

        # perform optimization step
        optimizer.step()

        # perform scheduler step
        scheduler.step()

        y_pred.extend(preds)
        y_true.extend(targets)

        if eval:
            train_y, train_loss = y_loss(y_true, y_pred, losses)
            train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

            if step % print_every_step == 0:
                eval_y, eval_loss = eval_op(model, eval_data_loader, loss_fn)
                eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

                if hasattr(eval_cb, '__call__'):
                    eval_loss_min = eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min)

    train_y, train_loss = y_loss(y_true, y_pred, losses)

    return train_y, train_loss, step, eval_loss_min

def eval_callback(epoch, epochs, output_path):
    def eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min):
        statement = ''
        statement += 'Epoch: {}/{}...'.format(epoch, epochs)
        statement += 'Step: {}...'.format(step)

        statement += 'Train Loss: {:.6f}...'.format(train_loss)
        statement += 'Train Acc: {:.3f}...'.format(train_score['acc'])

        statement += 'Valid Loss: {:.6f}...'.format(eval_loss)
        statement += 'Valid Acc: {:.3f}...'.format(eval_score['acc'])

        print(statement)

        if eval_loss <= eval_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                eval_loss_min,
                eval_loss))

            torch.save(model.state_dict(), output_path)
            eval_loss_min = eval_loss

        return eval_loss_min


    return eval_cb

In [ ]:
from torch import optim

optimizer = optim.Adam(pt_model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss()

step = 0
eval_loss_min = np.Inf
history = collections.defaultdict(list)


for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs... "):
    train_y, train_loss, step, eval_loss_min = train_op(
        model=pt_model,
        data_loader=train_data_loader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        scheduler=scheduler,
        step=step,
        print_every_step=EEVERY_EPOCH,
        eval=True,
        eval_cb=eval_callback(epoch, EPOCHS, OUTPUT_PATH),
        eval_loss_min=eval_loss_min,
        eval_data_loader=valid_data_loader,
        clip=CLIP)

    train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

    eval_y, eval_loss = eval_op(
        model=pt_model,
        data_loader=valid_data_loader,
        loss_fn=loss_fn)

    eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

    history['train_acc'].append(train_score['acc'])
    history['train_loss'].append(train_loss)
    history['val_acc'].append(eval_score['acc'])
    history['val_loss'].append(eval_loss)

Epochs... :   0%|          | 0/3 [00:00<?, ?it/s]

Training... :   0%|          | 0/542 [00:00<?, ?it/s]

##### Prediction

In [ ]:
def predict(model, comments, tokenizer, max_len=128, batch_size=32):
    data_loader = create_data_loader(comments, None, tokenizer, max_len, batch_size, None)

    predictions = []
    prediction_probs = []


    model.eval()
    with torch.no_grad():
        for dl in tqdm(data_loader, position=0):
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']

            # move tensors to GPU if CUDA is available
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)

            # compute predicted outputs by passing inputs to the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)

            # convert output probabilities to predicted class
            _, preds = torch.max(outputs, dim=1)

            predictions.extend(preds)
            prediction_probs.extend(F.softmax(outputs, dim=1))

    predictions = torch.stack(predictions).cpu().detach().numpy()
    prediction_probs = torch.stack(prediction_probs).cpu().detach().numpy()

    return predictions, prediction_probs

In [ ]:
test_sents = test['normalized_sents'].to_numpy()
preds, probs = predict(pt_model, test_sents, tokenizer, max_len=128)

print(preds.shape, probs.shape)

  0%|          | 0/590 [00:00<?, ?it/s]

(18860,) (18860, 10)


In [ ]:
preds

array([9, 5, 5, ..., 3, 4, 6])

In [ ]:
preds_label = [id2label[t] for t in preds]
preds_label[0:5]

['parvin', 'hafez', 'hafez', 'asad', 'asadi']

#####F1 Score and report

In [ ]:
y_test, y_pred = test['author'].values, preds

print(f'F1: {f1_score(y_test, y_pred, average="weighted")}')
print("--------------classification_report---------------")
print(classification_report(y_test, y_pred, target_names=label_list))

F1: 0.18183054145230343
--------------classification_report---------------
              precision    recall  f1-score   support

      khajoo       0.19      0.06      0.09      1886
        asad       0.17      0.23      0.19      1886
       bahar       0.18      0.21      0.19      1886
        feyz       0.20      0.20      0.20      1886
       asadi       0.28      0.41      0.33      1886
       hafez       0.18      0.26      0.21      1886
        jami       0.15      0.17      0.16      1886
       kamal       0.16      0.12      0.14      1886
     moulavi       0.22      0.09      0.13      1886
      parvin       0.18      0.17      0.17      1886

    accuracy                           0.19     18860
   macro avg       0.19      0.19      0.18     18860
weighted avg       0.19      0.19      0.18     18860



In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test,y_pred))

[[108 230 135 240 211 368 175 178 102 139]
 [ 42 427 178 172 363 210 164 122  41 167]
 [ 28 186 396 131 281 199 247 122  90 206]
 [ 61 231 157 370  91 361 230 155  83 147]
 [ 23 258 233  63 769  90 154 103  29 164]
 [ 79 231 181 217 104 495 202 149  80 148]
 [ 52 200 255 146 294 210 317 132  70 210]
 [ 74 261 182 222 209 282 201 229  86 140]
 [ 34 281 270 161 189 241 237 149 178 146]
 [ 53 242 263 138 199 292 202 131  48 318]]


####Preprocess data(add token, tokenizaiton, padding)

####Train and evaluation section

####Test

##Token classification

###GRU and LSTM

In [ ]:
class LSTMClassifier(nn.Module):

    def __init__(self,
        num_classes,
        batch_size=10,
        embedding_dim=100,
        hidden_dim=50,
        vocab_size=128):

        super(LSTMClassifier, self).__init__()

        initrange = 0.1

        self.num_labels = num_classes
        n = len(self.num_labels)
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.word_embeddings.weight.data.uniform_(-initrange, initrange)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True, bidirectional=True)  # !

        print("# !")

        bi_grus = torch.nn.GRU(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True, bidirectional=True)
        reverse_gru = torch.nn.GRU(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True, bidirectional=False)

        self.lstm.weight_ih_l0_reverse = bi_grus.weight_ih_l0_reverse
        self.lstm.weight_hh_l0_reverse = bi_grus.weight_hh_l0_reverse
        self.lstm.bias_ih_l0_reverse = bi_grus.bias_ih_l0_reverse
        self.lstm.bias_hh_l0_reverse = bi_grus.bias_hh_l0_reverse

        bi_output, bi_hidden = bi_grus()
        reverse_output, reverse_hidden = reverse_gru()

        print("# !")

        # self.classifier = nn.Linear(hidden_dim, self.num_labels[0])
        self.classifier = nn.Linear(2 * hidden_dim, self.num_labels[0])  # !


    def repackage_hidden(h):
        """Wraps hidden states in new Tensors, to detach them from their history."""

        if isinstance(h, torch.Tensor):
            return h.detach()
        else:
            return tuple(repackage_hidden(v) for v in h)


    def forward(self, sentence, labels=None):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds)  # lstm_out - 2 tensors, _ - hidden layer
        print(lstm_out[:,-1,:])
        tag_space = self.classifier(lstm_out[:,-1,:] + lstm_out[:,-1,:])  # !  # lstm_out[:,-1,:] - 1 tensor
        logits = F.log_softmax(tag_space, dim=1)
        loss = None
        if labels:
            loss = F.cross_entropy(logits.view(-1, self.num_labels[0]), labels[0].view(-1))
        return loss, logits

###Transformer-based model